In [1]:
import sklearn
from sklearn import datasets

iris = datasets.load_iris()
X_temp = iris.data
y_temp = iris.target

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X, y = X_temp[:, [0,1]], y_temp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
    
    
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
import numpy as np
print ('Training set mean:{:.2f} and standard deviation:{:.2f}'.format(np.average(X_train),np.std(X_train)))
print ('Testing set mean:{:.2f} and standard deviation:{:.2f}'.format(np.average(X_test),np.std(X_test)))

Training set mean:0.00 and standard deviation:1.00
Testing set mean:0.13 and standard deviation:0.71


In [ ]:
# প্লটিং লাইব্রেরি লোড করে নেই 
import matplotlib.pyplot as plt

# তিন প্রজাতির তিনটা আলাদা রং, মার্কার সহ 
colour_mk = [ ['red','s'], ['green','o'], ['blue','x']]
plt.figure('Training Data')

# লুপে ফেলে দিলাম, x এবং y এক্সিসে  
for i in range(len(colour_mk)):
    xs = X_train[:, 0][y_train == i]
    ys = X_train[:, 1][y_train == i]
    plt.scatter(xs, ys, c=colour_mk[i][0], marker=colour_mk[i][1])

# সাদা ব্যাকগ্রাউন্ড দরকার আমার, গুগল কোলাবে বাড়তি শেড দরকার নেই  
# plt.rcParams['axes.facecolor'] = 'white'
plt.style.use('default')
plt.grid(c='grey')

# প্লটিং প্যারামিটার 
plt.title('Training instances, after scaling')
plt.legend(iris.target_names)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.show()